#### 사용자 정의 모델 클래스
- 부모클래스 : nn.Module
- 필수오버라이딩 : 
    * _ _init_ _() : 모델 층 구성 즉, 설계
    * forward() : 순방 향학습 진행 코드 구현 
- 동적 모델
    * container 모듈 중 nn.ModuleList() 사용해서 동적으로 Layer 추가
        - forward 기능 미 제공 >> 순서, 연결성 X
        - layer 인스턴스 요소 사이에 연관성 없음 
        - layer 인스턴스 요소 인덱싱으로 접근 >>> 순서를 부여


In [1]:
# 모듈 로딩

import torch                                    # 텐서 관련 모듈
import torch.nn as nn                           # 인공신경망 관련 모듈
import torch.nn.functional as F                 # 손실함수 활성화함수 등 인공신경망 관련 함수 모듈
import torch.optim as optim                     # 최적화 관련 모듈(가중치, 절편 빠르게 찾아주는 알고리즘 (경사하강법))
from torchinfo import summary                   # 모델 구조 및 정보 관련 모듈
from torchmetrics.regression import *           # 회귀 성능 지표 관련 모듈 
from torchmetrics.classification import *       # 분류 성능 지표 관련 모듈

import pandas as pd                 # 데이터 파일 분석 관련 모듈
from sklearn.model_selection import train_test_split


In [2]:
# 랜덤 고정
torch.manual_seed(1)

# 텐서 저장 및 실행 위치 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

- 모델 설계 - 동적 모델 <hr>
    * 목표 : 은닉층의 개수가 동적인 모델
    * 조건
        + 입력층과 출력층 개수 동적 => 일력층의 입력값, 출력층의 출력값
        + 은닉충의 개수 동적 + 퍼셉트론 동적 => 은닉층의 개수, 은닉층 마다 퍼셉트론의 수

In [3]:
# ----------------------------------------------------------------------
# 모델 이름 : DynamicModel
# 부모 클래스 : nn.Module
# 모델의 메개변수 : in_in, out_out, h_ins=[], h_outs=[])  '**' >>> 키워드 파라미터 
# ----------------------------------------------------------------------

In [4]:
class DynamicModel(nn.Module):

    # 모델 구조 설계 함수 즉, 생성자 메서드
    def __init__(self,in_in, in_out, out_out, h_inout=[]):
        super().__init__()

        self.in_layer = nn.Linear(in_in, h_inout[0] if len(h_inout) else in_out)

        self.h_layers = nn.ModuleList()
        for idx in range(len(h_inout)-1):
            self.h_layers.append(nn.Linear(h_inout[idx], h_inout[idx+1]))
       
        self.out_layer = nn.Linear(h_inout[-1] if len(h_inout) else in_out, out_out)


    # 학습 진행 콜백 메서드
    def forward(self, x) :
        
        # 입력층
        y=self.in_layer(x)            # y = x1W1 + x2W2 + x3W3 + b
        y=F.relu(y)                   # 0 <= y 
        # y=F.relu(self.in_layer(x)) 
      
        # 은닉층
        for linear in self.h_layers:
            y = linear(y)
            y = F.relu(y)
            # y=F.relu(linear(y))

        # 출력층
        return self.out_layer(y)



In [5]:
# 모델 인스턴스 생성 >> 한 리스트로

h_inout = [10,30,50,70,90]
# for 문으로 idx로 뽑아서 
m1 = DynamicModel(in_in=3, in_out=5, out_out=2, h_inout=h_inout)

In [6]:
print(m1)

DynamicModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (h_layers): ModuleList(
    (0): Linear(in_features=10, out_features=30, bias=True)
    (1): Linear(in_features=30, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=70, bias=True)
    (3): Linear(in_features=70, out_features=90, bias=True)
  )
  (out_layer): Linear(in_features=90, out_features=2, bias=True)
)


In [7]:
# 모델 파라미터 확인
for name, param in m1.named_parameters():
    print(name, param.shape)

in_layer.weight torch.Size([10, 3])
in_layer.bias torch.Size([10])
h_layers.0.weight torch.Size([30, 10])
h_layers.0.bias torch.Size([30])
h_layers.1.weight torch.Size([50, 30])
h_layers.1.bias torch.Size([50])
h_layers.2.weight torch.Size([70, 50])
h_layers.2.bias torch.Size([70])
h_layers.3.weight torch.Size([90, 70])
h_layers.3.bias torch.Size([90])
out_layer.weight torch.Size([2, 90])
out_layer.bias torch.Size([2])


- 학습 진행

In [8]:
## 임시 데이터 생성성
dataTS =torch.FloatTensor([[1,3,5],[2,4,6],[3,5,7],[4,6,8]])    # 2D (4,3)
targetTS = torch.FloatTensor([[7,9],[8,10],[9,11],[10,12]])     # 2D (4,2)

In [9]:
# 모델 학습 
pre_y = m1(dataTS)

print(pre_y, pre_y.shape, sep='\n')

tensor([[ 0.1261, -0.0834],
        [ 0.1307, -0.0834],
        [ 0.1354, -0.0827],
        [ 0.1396, -0.0817]], grad_fn=<AddmmBackward0>)
torch.Size([4, 2])
